In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
          pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Problem Description

We have a dataset of movie reviews organized in text files, with the following folder structure:

## Folder structure
```
reviews
    |__ positive
        |__pos_1.txt
        |__pos_2.txt
        |__pos_3.txt
    |__ negative
        |__neg_1.txt
        |__neg_2.txt
        |__neg_3.txt
```  



## Task

We need to preprocess these reviews using `tf.data.Dataset` and perform the following transformations:

1. **Read Text Reviews**: Read the content of each text file and generate a label based on the folder name (`positive` or `negative`). The dataset should consist of tuples where each tuple contains the review text and its corresponding label.
   
2. **Filter Blank Reviews**: Exclude any text files that are blank. There are two blank files in this dataset.
   
3. **Shuffle Reviews**: Shuffle the dataset to randomize the order of the reviews.

Implement all of these transformations in a single line of code.


# Retrieve review file paths in a tensorflow dataset

In [2]:
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')


2024-07-25 02:03:14.104310: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 02:03:14.104446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 02:03:14.268753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
reviews_ds = tf.data.Dataset.list_files('/kaggle/input/reviews-data/reviews/*/*', shuffle=False)

In [4]:
for file in reviews_ds:
    print(file.numpy())

b'/kaggle/input/reviews-data/reviews/negative/neg_1.txt'
b'/kaggle/input/reviews-data/reviews/negative/neg_2.txt'
b'/kaggle/input/reviews-data/reviews/negative/neg_3.txt'
b'/kaggle/input/reviews-data/reviews/positive/pos_1.txt'
b'/kaggle/input/reviews-data/reviews/positive/pos_2.txt'
b'/kaggle/input/reviews-data/reviews/positive/pos_3.txt'


## Extract Review Text and Labels

- Extract the review text from each file.
- Extract the label from the folder name (`positive` or `negative`).


In [5]:
import os
def extract_review_and_label(file_path):
    return tf.io.read_file(file_path), tf.strings.split(file_path, os.path.sep)[-2]

In [6]:
reviews_ds_1 = reviews_ds.map(extract_review_and_label)
for review, label in reviews_ds_1:
    print("Review: ",review.numpy()[:50])
    print("Label: ",label.numpy())

Review:  b"Basically there's a family where a little boy (Jak"
Label:  b'negative'
Review:  b'This show was an amazing, fresh & innovative idea '
Label:  b'negative'
Review:  b''
Label:  b'negative'
Review:  b'One of the other reviewers has mentioned that afte'
Label:  b'positive'
Review:  b'A wonderful little production. <br /><br />The fil'
Label:  b'positive'
Review:  b''
Label:  b'positive'


## Filter blank reviews

In [7]:
reviews_ds_2 = reviews_ds_1.filter(lambda review, label: review!="")
for review, label in reviews_ds_2.as_numpy_iterator():
    print("Review: ",review[:50])
    print("Label: ",label)

Review:  b"Basically there's a family where a little boy (Jak"
Label:  b'negative'
Review:  b'This show was an amazing, fresh & innovative idea '
Label:  b'negative'
Review:  b'One of the other reviewers has mentioned that afte'
Label:  b'positive'
Review:  b'A wonderful little production. <br /><br />The fil'
Label:  b'positive'


# Perform map, filter and shuffle all in single line of code

In [8]:
final_ds = reviews_ds.map(extract_review_and_label).filter(lambda review, label: review!="").shuffle(3)
for review, label in final_ds.as_numpy_iterator():
    print("Review:",review[:50])
    print("Label:",label)

Review: b"Basically there's a family where a little boy (Jak"
Label: b'negative'
Review: b'This show was an amazing, fresh & innovative idea '
Label: b'negative'
Review: b'One of the other reviewers has mentioned that afte'
Label: b'positive'
Review: b'A wonderful little production. <br /><br />The fil'
Label: b'positive'
